In [2]:
import os
import torch
from training import train_model, hyperparameter_tuning
from model import AdvancedLSTM, LSTMEncoderOnly

In [3]:
file_path = os.getcwd() + "/final_data" # run from root
X_train, y_train = torch.load(file_path + "/train_sequences.pt", weights_only=True)
X_val, y_val = torch.load(file_path + "/val_sequences.pt", weights_only=True) 

print(f"Train sequences: {X_train.shape}, Targets: {y_train.shape}")

# Hyperparameter tuning
#best_params = hyperparameter_tuning(X_train, y_train, X_val, y_val, epochs=30, n_trials=25)

# Full training with best hyperparameters
print("\nTraining final model with best hyperparameters...")
model = LSTMEncoderOnly(input_dim=26, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2)
#model = TransformerEncoderOnly(input_dim=41, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2, nhead=4)
#adv_model = AdvancedLSTM(input_dim=26, hidden_dim=best_params['hidden_dim'], 
 #                           output_dim=1, num_layers=best_params['num_layers'], 
  #                          dropout=best_params['dropout'], num_fc_layers=best_params['num_fc_layers'])


train_model(
    model,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val, 
    learning_rate=1e-3,#best_params['learning_rate'],
    weight_decay=1e-5,#best_params['weight_decay'],
    batch_size=32,#best_params['batch_size'],
    epochs=50  # Full training
)

Train sequences: torch.Size([22472, 5, 26]), Targets: torch.Size([22472, 1])

Training final model with best hyperparameters...


Epoch  1/50: 100%|██████████| 703/703 [00:21<00:00, 32.83it/s, loss=0.4214, lr=5.05e-5]


Epoch 1 Training MSE: 0.6613
Epoch 1 validation RMSE: 2.8301
Epoch 1 validation MAE: 1.8185
Best model saved at epoch 1 with RMSE: 2.8301


Epoch  2/50: 100%|██████████| 703/703 [00:22<00:00, 31.79it/s, loss=0.2998, lr=8.15e-5]


Epoch 2 Training MSE: 0.3743
Epoch 2 validation RMSE: 2.8088
Epoch 2 validation MAE: 1.8015
Best model saved at epoch 2 with RMSE: 2.8088


...

Epoch 12/50: 100%|██████████| 703/703 [00:20<00:00, 34.58it/s, loss=0.2264, lr=0.000908]


Epoch 12 Training MSE: 0.3607
Epoch 12 validation RMSE: 2.7182
Epoch 12 validation MAE: 1.8719
Best model saved at epoch 12 with RMSE: 2.7182


Epoch 13/50: 100%|██████████| 703/703 [00:23<00:00, 30.10it/s, loss=0.1200, lr=0.000959]


Epoch 13 Training MSE: 0.3602


KeyboardInterrupt: 